In [47]:
from IPython.display import display, Markdown, clear_output
import utils
import ipywidgets as ipw
import widgets

## Sample preparation

In [48]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
ACTIONS = ["SPUTTERING", "ANNEALING", "DEPOSITION", "DOSING"]

In [49]:
sample_selector = widgets.ObjectSelectionWidget("Sample")
sample_selector.load_dropdown_box()

sample_history = ipw.Accordion()

add_process = utils.Button(
    description = 'Add protocol process', disabled = False, button_style = '', 
    tooltip = 'Add process with protocol', layout = ipw.Layout(width = '150px', height = '25px')
)

add_process_step = utils.Button(
    description = 'Add process step', disabled = False, button_style = '', 
    tooltip = 'Add process step', layout = ipw.Layout(width = '150px', height = '25px')
)

process_step_buttons = ipw.HBox([add_process, add_process_step])

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def get_sample_processes(sample_identifier):
    sample_processes = {}
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_identifier)
    objects_stack = [(sample_object, None)]
    
    while objects_stack:
        current_object, child_object = objects_stack.pop()
        current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid)
        
        if current_object_data["type"] == "SAMPLE_PROCESS" and current_object_data["permId"] not in sample_processes:
            process_actions = []
            process_observables = []
            process_children = current_object_data["children"]
            for child_identifier in process_children:
                child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier)
                if child_object_data["type"] in ACTIONS:
                    process_actions.append(child_object_data["permId"])
                elif child_object_data["type"] == "OBSERVABLE":
                    process_observables.append(child_object_data["permId"])
            
            if process_actions or process_observables:   
                sample_processes[current_object_data["permId"]] = {
                    "actions": process_actions,
                    "observables": process_observables
                }
        
        for parent_identifier in current_object.parents:
            parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
            objects_stack.append((parent_object, current_object))
            
    return sample_processes

def get_sample_processes_widgets(sample_processes):
    process_steps_accordion = ipw.Accordion()
    process_steps_accordion_items = []
    protocol_process_items = {}
    for process_identifier in sample_processes:
        process_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_identifier)
        
        process_actions_accordion = ipw.Accordion()
        process_actions_accordion_items = []
        process_observables_accordion = ipw.Accordion()
        process_observables_accordion_items = []
        
        for process_action_identifier in sample_processes[process_identifier]["actions"]:
            process_action_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_action_identifier)
            process_actions_accordion.set_title(len(process_actions_accordion_items), process_action_data["props"]["$name"])
            process_actions_accordion_items.append(ipw.HBox())
        process_actions_accordion.children = process_actions_accordion_items
        
        for process_observable_identifier in sample_processes[process_identifier]["observables"]:
            process_observable_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_observable_identifier)
            process_observables_accordion.set_title(len(process_observables_accordion_items), process_observable_data["props"]["$name"])
            process_observables_accordion_items.append(ipw.HBox())
        process_observables_accordion.children = process_observables_accordion_items
        
        process_actions_label = ipw.Label(value = "Actions")
        process_observables_label = ipw.Label(value = "Observables")
        process_step_items = ipw.VBox([process_actions_label, process_actions_accordion, process_observables_label, process_observables_accordion])
        
        follows_protocol = False
        for process_parent_identifier in process_data["parents"]:
            process_parent_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_parent_identifier)
            if process_parent_data["type"] == "SAMPLE_PROCESS":
                follows_protocol = True
                protocol_process_identifier = process_parent_data["permId"]
                break
        
        if follows_protocol:
            process_step_details = {"name": process_data["props"]["$name"], "items": process_step_items}
            if protocol_process_identifier in protocol_process_items:
                protocol_process_items[protocol_process_identifier].append(process_step_details)
            else:
                protocol_process_items[protocol_process_identifier] = [process_step_details]
        else:
            process_steps_accordion.set_title(len(process_steps_accordion_items), process_data["props"]["$name"])
            process_steps_accordion_items.append(process_step_items)
    
    for protocol_process_identifier, protocol_process_steps_details in protocol_process_items.items():
        protocol_process_data = utils.get_openbis_object_data(OPENBIS_SESSION, protocol_process_identifier)
        protocol_process_accordion = ipw.Accordion()
        protocol_process_accordion_list = []
        for protocol_process_step_details in protocol_process_steps_details:
            protocol_process_accordion.set_title(len(protocol_process_accordion_list), protocol_process_step_details["name"])
            protocol_process_accordion_list.append(protocol_process_step_details["items"])
        protocol_process_accordion.children = protocol_process_accordion_list
        
        process_steps_accordion.set_title(len(process_steps_accordion_items), protocol_process_data["props"]["$name"])
        process_steps_accordion_items.append(protocol_process_accordion)
    
    process_steps_accordion.children = process_steps_accordion_items
        
    return process_steps_accordion

In [51]:
sample_processes = get_sample_processes("20250303102131254-33252")
sample_processes_widgets = get_sample_processes_widgets(sample_processes)

In [52]:
display(Markdown("## Select sample"))
display(sample_selector)

display(Markdown("## Sample history"))
display(sample_processes_widgets)

display(Markdown("## New sample preprocessing steps"))
display(process_step_buttons)
display(save_close_buttons_hbox)
display(increase_buttons_size)

## Select sample

ObjectSelectionWidget(children=(VBox(children=(Dropdown(description='Sample', layout=Layout(width='982px'), op…

## Sample history

Accordion(children=(VBox(children=(Label(value='Actions'), Accordion(children=(HBox(),), _titles={'0': 'Anneal…

## New sample preprocessing steps